In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your custom dataset paths
train_dir = r'C:\Users\User\Downloads\DataSet\Train'
test_dir = r'C:\Users\User\Downloads\DataSet\Test'

# Define image dimensions and batch size
img_width, img_height = 28, 28
batch_size = 32

# Data augmentation for training data (optional but recommended)
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'  # Set to 'categorical' for multiple classes
)

# Load and preprocess the test data
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'  # Set to 'categorical' for multiple classes
)

# Define the number of classes in your dataset
num_classes = len(train_generator.class_indices)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Dropout layer to reduce overfitting
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs = 40  # You can adjust the number of epochs as needed
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'Test accuracy: {test_acc}')

Found 400 images belonging to 10 classes.
Found 100 images belonging to 10 classes.
Epoch 1/40
12/12 [==============================] - 3s 128ms/step - loss: 2.3104 - accuracy: 0.1087 - val_loss: 2.2987 - val_accuracy: 0.1042
Epoch 2/40
12/12 [==============================] - 1s 104ms/step - loss: 2.2956 - accuracy: 0.1114 - val_loss: 2.2896 - val_accuracy: 0.1562
Epoch 3/40
12/12 [==============================] - 1s 102ms/step - loss: 2.2815 - accuracy: 0.0978 - val_loss: 2.2607 - val_accuracy: 0.1667
Epoch 4/40
12/12 [==============================] - 1s 114ms/step - loss: 2.2587 - accuracy: 0.1250 - val_loss: 2.2389 - val_accuracy: 0.1458
Epoch 5/40
12/12 [==============================] - 1s 105ms/step - loss: 2.2097 - accuracy: 0.2092 - val_loss: 2.2671 - val_accuracy: 0.1146
Epoch 6/40
12/12 [==============================] - 1s 105ms/step - loss: 2.1875 - accuracy: 0.1712 - val_loss: 2.1996 - val_accuracy: 0.0938
Epoch 7/40
12/12 [==============================] - 1s 103ms/ste

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 1, 1, 128)        

In [8]:
print("The model has successfully trained")
model.save('custom.h5')
print("Saving the model as custom.h5")

The model has successfully trained
Saving the model as custom.h5


In [ ]:
from keras.models import load_model
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('custom.h5')

def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB to grayscale
    img = img.convert('L')
    # Convert grayscale image to three-channel (RGB)
    img = Image.merge('RGB', (img, img, img))
    img = np.array(img)
    # Reshape for model normalization
    img = img.reshape(1, 28, 28, 3)  # Use 3 channels
    img = img / 255.0
    # Predict the digit
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=200, height=200, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Analyzing..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Search", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=tk.W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.canvas.bind("<Button-1>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        hd = self.canvas.winfo_id()  # to fetch the handle of the canvas
        rect = win32gui.GetWindowRect(hd)  # to fetch the edges of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')

    def start_pos(self, event):
        self.x = event.x
        self.y = event.y

    def draw_lines(self, event):
        x1, y1 = (event.x - 1), (event.y - 1)
        x2, y2 = (event.x + 1), (event.y + 1)
        self.canvas.create_oval(x1, y1, x2, y2, fill='white', width=5)

app = App()
app.mainloop()